In [1]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
# This is the path where you want to search
path = r'NER_PRECEDENTE_EXPORT'

# this is the extension you want to detect
extension = '.csv'
file_name_path1 = []
for root, dirs_list, files_list in os.walk(path):
    for file_name in files_list:
        if os.path.splitext(file_name)[-1] == extension:
            file_name_path = os.path.join(root, file_name)
#             print(file_name)
#             print(file_name_path)   # This is the full path of the filter file
            file_name_path1.append(file_name_path)

In [3]:
len(file_name_path1)

29412

In [4]:
# Une todos os arquivos.

frames = []
for all_files in file_name_path1:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
df_prec = pd.concat(frames).reset_index(drop=True)

In [5]:
df_prec.to_csv("df_prec.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
df_prec['Token'] = df_prec['Token'].astype('str')

In [6]:
df_prec.head(30)

,Token,Tag
0,Súmula,B_Classe - Súmula
1,,I_Classe - Súmula
2,284,B_Número
3,/,I_Número
4,STF,B_Corte
5,Rcl,B_Classe - Writ
6,,I_Classe - Writ
7,11.022,B_Número
8,-,I_Número
9,ED,B_Tipo


In [7]:
# Tags de precedente
list(df_prec.Tag.unique())

['B_Classe - Súmula',
 'I_Classe - Súmula',
 'B_Número',
 'I_Número',
 'B_Corte',
 'B_Classe - Writ',
 'I_Classe - Writ',
 'B_Tipo',
 'I_Tipo',
 'O',
 'B_Relator',
 'I_Relator',
 'B_Data de Publicação',
 'B_Origem',
 'B_Classe - Recursal',
 'I_Classe - Recursal',
 'I_Origem',
 'B_Data de Julgamento',
 'I_Corte',
 'B_Classe - Penal originária',
 'I_Classe - Penal originária',
 'B_Classe - Outros',
 'I_Classe - Outros',
 'I_Data de Julgamento',
 'B_Classe - Constitucional',
 'I_Classe - Constitucional',
 'I_Data de Publicação',
 'B_Pessoa - Não Relator',
 'I_Pessoa - Não Relator']

# Ideia para separação de sentenças:

-- Obter uma tag inicial que começa com B_, a sentença dura até que 'I_(fim da tag inicial)' se altere

In [8]:
extension = 'csv'
all_filenames = [i for i in glob.glob('NER/*/**/***/****/*****.{}'.format(extension))]

In [11]:
len(all_filenames)

2239

In [12]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ'] # Altera a primeira e ultima Tag desse csv
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
combined_csv = pd.concat(frames).reset_index(drop=True)
combined_csv.to_csv("combined_csv.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
combined_csv['Token'] = combined_csv['Token'].astype('str')

In [17]:
combined_csv[combined_csv.Tag.str.endswith('Precedente')].head(20)

,Token,Tag
66,ADI,B_Precedente
67,,I_Precedente
68,2.588,I_Precedente
69,/,I_Precedente
70,DF,I_Precedente
71,",",I_Precedente
837,RECURSO,B_Precedente
838,,I_Precedente
839,EXTRAORDINÁRIO,I_Precedente
840,,I_Precedente


In [29]:
combined_csv.iloc[68-5:68+5]

,Token,Tag
63,,O
64,da,O
65,,O
66,ADI,B_Precedente
67,,I_Precedente
68,2.588,I_Precedente
69,/,I_Precedente
70,DF,I_Precedente
71,",",I_Precedente
72,,O


In [58]:
df_prec.head(5)

,Token,Tag
0,RTJ,B_Classe - Outros
1,,I_Classe - Outros
2,87/758,B_Número
3,HC,B_Classe - Writ
4,,I_Classe - Writ


In [25]:
listadi = df_prec[df_prec.Token.str.startswith('ADI')].index.values
i = 2
df_prec.iloc[listadi[i]-3:listadi[i]+5]

,Token,Tag
4108,de,I_Relator
4109,,I_Relator
4110,Faria,I_Relator
4111,ADI,B_Classe - Constitucional
4112,,I_Classe - Constitucional
4113,4.643,B_Número
4114,",",I_Número
4115,,O


In [26]:
df_prec[df_prec.Token=='2.588']

,Token,Tag
104520,2.588,B_Número
211158,2.588,B_Número
367439,2.588,B_Número
382803,2.588,B_Número


In [28]:
df_prec.iloc[104520-5:104520+5]

,Token,Tag
104515,Cármen,I_Relator
104516,,I_Relator
104517,Lúcia,I_Relator
104518,ADI,B_Classe - Constitucional
104519,,I_Classe - Constitucional
104520,2.588,B_Número
104521,/,I_Número
104522,DF,B_Origem
104523,AÇÃO,B_Classe - Constitucional
104524,,I_Classe - Constitucional


In [63]:
# Conferindo se nao pode existir o caso onde uma Tag que começa com 'I' é seguida de outra Tag que começa
# com 'I'
lista = []
for i in range(len(df_prec)):
    if df_prec.Tag.iloc[i].startswith('I') == True and df_prec.Tag.iloc[i] != df_prec.Tag.iloc[i+1] == True:
        lista = df_prec.Tag.iloc[i+1]
#         print(df_prec.Tag.iloc[i+1])
lista

[]